### LAB01B - Introduction to Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to interact with Large Language Models. We'll complete the following steps:

1) Import general dependencies.
2) Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI. 
3) Instantiate a Kernel to add LLM capabilities to your application.
4) Connect the Kernel to a Completion model to run your first semantic function.

**Important:** We're a using a method to automatically read sensitive parameters from a `.env` file. Therefore, please ensure that you have a `.env` file in the same directory of this notebook with the following parameters:

``` python
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR MODEL DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"
```
Optionally, you can update the values and run the cell below to create your `.env` file. Please note the deployment name as we'll use both Text Completion and Chat Completion in this LAB.

In [ ]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR TEXT COMPLETION DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"

#### Step 1 - Import Dependencies

**Semantic Kernel** is a SDK currently available in C#, Java and Python. However, each language support different at the current stage. Please visit [supported languages](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) to check which features are supported per language.

In this lab, we'll use Python SDK which can be obtained in Pypi repository.

**Important:** if you get an error running the code below, make sure that you've completed the setup process. Optionally, you can run `pip install semantic_kernel` in your terminal or `!python -m pip install semantic_kernel` in a Python code cell.

In [ ]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
#The other dependencies will be imported directly in the step where they are used.
import semantic_kernel as sk

#### Step 2 - Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI

In this step, we'll connect to an **Azure OpenAI Service** and use a Text Completion model. 
The Semantic Kernel also supports **OpenAI**. Other AI Services, like **Hugging Face** will be supported shortly. You can visit this [link](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages#ai-service-endpoints) to check the most updated list.

In this example, we're using **AzureTextCompletion** module to interact with Text Completion models from Azure OpenAI, such as `gpt-35-turbo-instruct` or `text-davinci-003`. This is suitable for text generation activities. We are also using a module to control the behavior of the requests, defining parameters like Temperature, max_tokens and even how many responses we want to generate.

In [ ]:
#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion
#Importing the module that contains the settings for the Azure Text Completion service
from semantic_kernel.connectors.ai import CompleteRequestSettings

# Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# Create the Azure Text Completion service connector
azure_text_completion = AzureTextCompletion(deployment, endpoint, api_key)

# Create the settings to control the behavior to the requests of Azure Text Completion service
request_settings = CompleteRequestSettings(
    max_tokens=20,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=2
)

Now that we have all set, let's run a simple prompt to generate taglines using the Text Completion model. Please note that the number of results, response lenght and temperature are controlled by the **CompleteRequestSettings**.

In [ ]:
prompt = "Write tagline for a Telecom company."

#Calling the Azure Text Completion service using Semantic Kernel SDK and AI Service Connector
results = await azure_text_completion.complete_async(prompt, request_settings)

#Printing out the results
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

Now that we know how to use the SDK to run a prompt, let's start building out our Call Center Analytics Solution. We have already explored how to use prompts to summarize and extract relevant information from call center conversations. As our next step, let's run them in Semantic Kernel. We'll explore different methods.

We'll use the conversation below as an example. If you prefer, just change it.

In [ ]:
#Example of a call center conversation between a customer and an agent. You can replace by your own example
input = """
Agente: Hola, bienvenido al servicio de atención al cliente de Telecom. Mi nombre es Juan, ¿en qué puedo ayudarlo?
Cliente: Hola, Juan. Estoy llamando porque tengo problemas con mi plan de datos móviles. Es muy lento y no puedo navegar por internet ni usar mis aplicaciones.
Agente: Lo siento mucho por las molestias, señor. ¿Me puede decir su número de teléfono y su nombre completo, por favor?
Cliente: Sí, claro. Mi número es 011-4567-8910 y mi nombre es Martín Pérez.
Agente: Gracias, señor Pérez. Voy a verificar su plan y su consumo de datos. Un momento, por favor.
Cliente: Está bien, gracias.
Agente: Señor Pérez, he revisado su plan y veo que tiene contratado el plan básico de 2 GB de datos por mes. ¿Es correcto?
Cliente: Sí, así es.
Agente: Bueno, le informo que ha consumido el 90% de su límite de datos y que le quedan solo 200 MB disponibles hasta el final del mes. Por eso su velocidad de navegación se ha reducido.
Cliente: ¿Qué? ¿Cómo es posible? Yo apenas uso el internet en mi celular. Solo reviso mi correo y mis redes sociales de vez en cuando. No veo videos ni descargo archivos grandes.
Agente: Entiendo, señor Pérez. Pero tenga en cuenta que algunas aplicaciones consumen datos en segundo plano, sin que usted se dé cuenta. Por ejemplo, las actualizaciones automáticas, las copias de seguridad, el GPS, etc.
Cliente: Bueno, pero eso no me lo explicaron cuando contraté el plan. Me dijeron que con 2 GB tendría suficiente para todo el mes. Me siento engañado.
Agente: Le pido disculpas, señor Pérez. No era nuestra intención engañarlo. Le ofrezco una solución: si quiere, puede cambiar su plan a uno superior, con más GB de datos y mayor velocidad. Así podrá disfrutar de una mejor experiencia de navegación.
Cliente: ¿Y cuánto me costaría eso?
Agente: Tenemos una oferta especial para usted. Por solo 10 pesos más al mes, puede acceder al plan premium de 5 GB de datos y velocidad 4G. ¿Le interesa?
Cliente: Mmm, no sé. ¿No hay otra opción? ¿No pueden darme más velocidad sin cobrarme más?
Agente: Lo siento, señor Pérez. Esa es la única opción que tenemos disponible. Si no cambia su plan, tendrá que esperar hasta el próximo mes para recuperar su velocidad normal. O puede comprar un paquete adicional de datos, pero le saldría más caro que cambiar de plan.
Cliente: Bueno, déjeme pensarlo. ¿Puedo llamar más tarde para confirmar?
Agente: Claro, señor Pérez. Puede llamar cuando quiera. El número es el mismo que marcó ahora. ¿Hay algo más en lo que pueda ayudarlo?
Cliente: No, eso es todo. Gracias por su atención.
Agente: Gracias a usted, señor Pérez. Que tenga un buen día. Adiós.
"""

Let's run the same prompt for "Problem Summarization". Note that we're using python strings to append the Conversation to our prompt. As you run this cell, we should see a summary similar to what we saw in Azure AI Studio. 

In [ ]:
#Initially, let's use a prompt similar to what we have tested in the AI Studio.
prompt = f"""
Summarize the problem described below. 
Be brief, yet informative.
Try to capture in the summary:
- The problem description;
- The product customer is complaining (if available);
- How Agente, handled the call;
- The outcome of the call. 

[INPUT]
{input}
[END INPUT]
"""

# Create the settings to control the behavior to the requests of Azure Text Completion service
request_settings = CompleteRequestSettings(
    max_tokens=200,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=1
)

#Calling the Azure Text Completion service using Semantic Kernel SDK and AI Service Connector
result = await azure_text_completion.complete_async(prompt, request_settings)

print(f"Result: {result}")

#### Step 3 - Instantiate a Kernel to add LLM capabilities to your application
The core of Semantic Kernel is it's Kernel. It's responsible to transform an User Input (or ASK) into a response by orchestrating Plugins. It's also what makes Semantic Kernel shines as it can transform existing applications in AI applications powered by LLMs. 

In this step, we'll instantiate a simple Kernel with no additonal options. Optionally, we could define Logging, Memory and the set of skills we want to use. 

In [ ]:
#Instantiating the kernel
kernel = sk.Kernel()


#Example of how to instantiate the kernel with a different log level
#logger = sk.NullLogger()
#kernel_with_logging = sk.Kernel(log=logger)

#### Step 4 - Connect the Kernel to a Completion model to run your first semantic function.
Now that we have our kernel, we can start adding capabilities to it. We already saw how to use the SDK and connectors to the AI services to run prompts. Now, let's see how can we use the Kernel to transform a prompt into a semantic function able to run Text Completion activities using a LLM.

The first step is to add to our kernel the Text Completion model that we already have defined before. We can add multiple models and give then names (or nicknames) to later decide which ones we want to use and define programmatically which one is the default.

You can also add Chat Completion models, like GPT-4 and GPT-3.5-Turbo, as per the commented code.

In [ ]:
# Add the text completion service to the kernel
kernel.add_text_completion_service("Text Completion", 
                                   AzureTextCompletion(deployment, endpoint, api_key))

#We can add several models of the same type to the kernel
#kernel.add_text_completion_service("Other Text Completion",
#                                   AzureTextCompletion("text-curie-001", endpoint, api_key))

#We can add other models types to the kernel, like ChatCompletions (GPT-35-Turbo, GPT4)
#from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
#kernel.add_chat_completion_service("ChatCompletion - GPT-4",
#                                   AzureChatCompletionCompletion("text-curie-001", endpoint, api_key))

#kernel.set_default_text_completion_service("Text Completion")

With the kernel connected to a large language model from Azure OpenAI Service, we can start using it to run Text Completion operations. The best way to make our prompts reusable is to create a **Semantic Function**. You can create and add one to the Kernel based on a **Prompt**. This is simple a textual description, or instruction, of what we want the function to do. Optionally, we can add parameters, like `{{$input}}`, to inject values in runtime.

We'll deep dive in Semantic Functions later and now we'll demonstrate a simplified way to create and run Semantic Function **inline**. With this method we can start structuring our application and transforming Prompts into functions. Please note that as we define the function we can also control the behavior of how we expect the kernel to run it, for example, controlling the max number of tokens in the response and the temperature. This we'll ensure that the function we'll run consistenly.

Therefore, in this step, we'll create an **inline semantic function** to classify a Telecom problem in different categories, like Mobile Internet, Fixed Internet, etc. This is based on the same example we run in Azure AI Studio and is one of the things we need to complete in our Call Center Analytics Solution.

In [ ]:
# Set the prompt for categorizing based on the problem description
prompt = """
Problem Description: {{$input}}
Classify the problem description above as one of these categories: Fixed Internet, Mobile Internet, TV, Landline, Billing.
Only write the output category with no extra text.
Only write one category per problem description.
"""

# Create a semantic function for classifying the problem description
classify_call_function = kernel.create_semantic_function(prompt, 
                                                         max_tokens=10,
                                                         temperature=0.5)

Now that our Kernel has a new function to Categorize problems, we'll test it based on a problem description. We expected as a result one of the classifications previously defined in the prompt.

In [ ]:
# Call the classify function with a problem description
classification = classify_call_function(input)

# Print the result
print(f"Result: {classification}")

Step 5 - Running Chat Completions

In [ ]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR CHAT COMPLETION DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"

We can also run Chat Completion Operations with Semantic Kernel. Let's see a simple example

In [ ]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai import CompleteRequestSettings

#Instantiating the kernel
kernel = sk.Kernel()

# Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service("Chat Completion", 
                        AzureChatCompletion(deployment, endpoint, api_key))

#Define a system message to model how the model should behave
system_message = """
You're an AI assistant that helps Telecom company to extract valuable information from their conversations by creating JSON files for each conversation transcription you receive. You always try to extract and format as a JSON:
1. Customer Name [name]
2. Customer Contact Phone [phone]
3. Main Topic of the Conversation [topic]
4. Customer Sentiment (Neutral, Positive, Negative)[sentiment]
5. How the Agent Handled the Conversation [agent_behavior]
6. What was the FINAL Outcome of the Conversation [outcome]
7. A really brief Summary of the Conversation [summary]

Only extract information that you're sure. If you're unsure, write "Unknown/Not Found" in the JSON file.
"""

#Prepare the prompt template to be used by the semantic function
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=500, temperature=0.7, top_p=0.5
)
prompt_template = sk.ChatPromptTemplate(
    "{{$input}}", kernel.prompt_template_engine, prompt_config
)

#Ensure our prompt template have at least a System Message
prompt_template.add_system_message(system_message)
#Optionally, we can load user and assistant messages to use as Few Shot examples
prompt_template.add_user_message("Hello, how are you?")
prompt_template.add_assistant_message("Hello! I can help you to extract valuable information from Call Center conversations.")

extract_function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
extract_fn = kernel.register_semantic_function("CallCenter", "ChatExtract", extract_function_config)
context_vars = sk.ContextVariables()
try:
    context_vars["input"] = input("Enter a Conversation Transcription:> ")
    result = await kernel.run_async(extract_fn, input_vars=context_vars)
    # Print the result
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")   
    exit()